In [ ]:
import numpy as np
import polars as pl
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import json
import re

# Импортируем классы из gpt.py
from gpt import (
    CharTokenizer, 
    SimpleTextDataset, 
    collate, 
    Transformer, 
    model_memory_size, 
    fix_seed
)

In [ ]:
fix_seed(42)

# DATA PREPARATION

In [ ]:
# with open("full_geralt.txt", "r") as file:
#     raw_content = file.read()

# corpus = []
# for line in raw_content.split("\n"):
#     cleaned_line = line.strip("\n")
#     if not re.search(r"\w", cleaned_line):
#         continue
#     corpus.append(cleaned_line)

names = pl.read_parquet("names.parquet")
surnames = pl.read_parquet("surnames.parquet")

def get_persons(names: pl.DataFrame, surnames: pl.DataFrame, n: int = 100) -> list[str]:
    persons = []
    for _ in range(n):
        sex = np.random.choice(["m", "f"]).item()
        name = names.filter(pl.col("gender") == sex).sample(1).select("text").item()
        surname = surnames.filter(pl.col("gender") == sex).sample(1).select("text").item()
        persons.append(f"{name} {surname}")
    return persons

corpus = get_persons(names, surnames, 10_000)
# corpus[:10]

In [ ]:
tokenizer = CharTokenizer().fit(corpus)
# tokenizer.vocab

In [ ]:
# save tokenizer vocab to json
with open("tokenizer_vocab.json", "w", encoding="utf-8") as f:
    json.dump(tokenizer.vocab, f, ensure_ascii=False, indent=2)

In [ ]:
VOCAB_SIZE = len(tokenizer.vocab)
BATCH_SIZE = 1024
MAX_SEQ_LEN = 200
N_LAYERS = 6
EMBEDDING_SIZE = 128
NUM_HEADS = 8
NUM_KV_GROUPS = 2
NUM_EXPERTS = 16
NUM_EXPERTS_PER_TOKEN = 2
HEAD_EMBEDDING_SIZE = EMBEDDING_SIZE // NUM_HEADS
FCCN_HIDDEN_SIZE = EMBEDDING_SIZE * 4
n_epoch = 20

In [ ]:
dataset = SimpleTextDataset(
    corpus=corpus,
    fitted_tokenizer=tokenizer,
    max_seq_length=MAX_SEQ_LEN,
)
dataloader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate
)
# next(iter(dataloader)).shape

In [ ]:
model = Transformer(
    vocab_size=VOCAB_SIZE,
    n_layers=N_LAYERS,
    embedding_size=EMBEDDING_SIZE,
    num_heads=NUM_HEADS,
    num_kv_groups=NUM_KV_GROUPS,
    num_experts=NUM_EXPERTS,
    num_experts_per_token=NUM_EXPERTS_PER_TOKEN,
    head_embedding_size=HEAD_EMBEDDING_SIZE,
    fcnn_hidden_size=FCCN_HIDDEN_SIZE,
    dropout=0.15,
)

optimizer = Adam(model.parameters(), lr=4e-3)
loss_func = nn.CrossEntropyLoss(reduction='none')

epoch_loss = []
device = "cuda:1"
model.to(device)
model.train()

In [ ]:
for i in range(n_epoch):
    losses = []
    print(f'Epoch {i + 1}')
    for x in tqdm(dataloader):
        curr_x = x[:, :-1]
        next_x = x[:, 1:].clone()
        next_x[(curr_x == 0) | (curr_x == 4)] = 0
        
        curr_x = curr_x.to(device)
        next_x = next_x.to(device)
        
        logits = model(curr_x)
        token_losses = loss_func(logits.transpose(1, 2), next_x.to(torch.long))
        loss = token_losses.sum() / (token_losses > 0).sum()
        losses.append(loss.item())
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    epoch_loss.append(np.mean(losses))
    print(f'Loss: {epoch_loss[-1]}')


In [ ]:
print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.4f} MB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.4f} MB")

In [ ]:
torch.save(model.state_dict(), "my_gpt_weights.pt")

In [ ]:
from decoding_strategies_over_custom_gpt import GenerativeModel

In [ ]:
gen_model = GenerativeModel(
    vocab_size=VOCAB_SIZE,
    n_layers=N_LAYERS,
    embedding_size=EMBEDDING_SIZE,
    num_heads=NUM_HEADS,
    num_kv_groups=NUM_KV_GROUPS,
    num_experts=NUM_EXPERTS,
    num_experts_per_token=NUM_EXPERTS_PER_TOKEN,
    head_embedding_size=HEAD_EMBEDDING_SIZE,
    fcnn_hidden_size=FCCN_HIDDEN_SIZE,
    dropout=0.15,
)

In [ ]:
gen_model.load_state_dict(model.state_dict())
gen_model.to(device)

In [ ]:
print("Generation Check")
test_prompt = list(tokenizer.vocab.values())[10]
print(f"Input: {test_prompt}")
print("Output:", gen_model.generate(test_prompt, tokenizer, device=device, max_new_tokens=20))